In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.optimize import fsolve, root
from scipy.stats import norm, beta as beta_dist
from scipy.integrate import quad
from IPython.display import Image, display, Markdown

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'figure.dpi': 130, 'font.size': 12, 'axes.titlesize': 'large',
    'axes.labelsize': 'medium', 'xtick.labelsize': 'small', 'ytick.labelsize': 'small'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 4: Macroeconomic Models
## Chapter 4.1: Foundational Job Search and Matching Models

### Table of Contents
1.  [The McCall Job Search Model](#1.-The-McCall-Job-Search-Model)
2.  [The Diamond-Mortensen-Pissarides (DMP) Matching Model](#2.-The-Diamond-Mortensen-Pissarides-(DMP)-Matching-Model)
    *   [2.1 The Model Environment and Bellman Equations](#2.1-The-Model-Environment-and-Bellman-Equations)
    *   [2.2 Solving for the General Equilibrium](#2.2-Solving-for-the-General-Equilibrium)
    *   [2.3 The Beveridge Curve](#2.3-The-Beveridge-Curve)
3.  [On-the-Job Search and Wage Dispersion](#3.-On-the-Job-Search-and-Wage-Dispersion)
    *   [3.1 The Burdett-Mortensen Model](#3.1-The-Burdett-Mortensen-Model)
    *   [3.2 The Equilibrium Wage Distribution](#3.2-The-Equilibrium-Wage-Distribution)
4.  [Assortative Matching](#4.-Assortative-Matching)
5.  [Chapter Summary](#5.-Chapter-Summary)
6.  [Exercises](#6.-Exercises)

### Introduction
This chapter explores foundational models of the labor market, focusing on the frictions that give rise to unemployment and determine the flow of workers between jobs. We will analyze three canonical frameworks:
1.  **The McCall Job Search Model**: A partial equilibrium model of an individual's decision to accept a job offer, which gives rise to the concept of a **reservation wage**.
2.  **The Diamond-Mortensen-Pissarides (DMP) Matching Model**: A general equilibrium model of the entire labor market that explains how frictions in the matching process determine the equilibrium rates of unemployment and vacancies.
3.  **The Burdett-Mortensen Model**: A model of on-the-job search that generates wage dispersion as an equilibrium outcome.
4.  **Assortative Matching**: A framework for understanding who matches with whom when both workers and firms are heterogeneous.

### 1. The McCall Job Search Model

The McCall model, first formulated by John McCall in 1970, is a foundational model of microeconomic job search. It elegantly captures the trade-off an unemployed individual faces: accept a known wage offer now, or reject it in the hope of receiving a better offer in the future, while foregoing income and incurring search costs. This decision-making process under uncertainty is a classic optimal stopping problem.

The model's central prediction is that the worker's optimal policy is defined by a **reservation wage**, $w^*$. This is a threshold: any offer at or above $w^*$ is accepted, and any offer below it is rejected. The reservation wage is the precise value that makes the worker indifferent between the long-term value of accepting the job and the expected long-term value of continuing to search. It is the solution to the equation:
$$ \frac{w^*}{1-\beta} = c + \beta E[\max(\frac{w'}{1-\beta}, \frac{w^*}{1-\beta})] $$
where $c$ is the flow value of unemployment (benefits), $\beta$ is the discount factor, and the expectation is over the distribution of future wage offers $w'$.

In [ ]:
sec("Solving for the McCall Reservation Wage")
# --- Model Parameters ---
# BETA: The discount factor, representing the agent's patience. A value of 0.96 implies an annual discount rate of about 4%.
# C: The flow value of unemployment, including unemployment benefits and the value of leisure.
# WAGE_DIST_MEAN, WAGE_DIST_STD: The mean and standard deviation of the normal distribution from which wage offers are drawn.
BETA, C = 0.96, 6.0
WAGE_DIST_MEAN, WAGE_DIST_STD = 10, 2.5
w_grid = np.linspace(WAGE_DIST_MEAN - 4*WAGE_DIST_STD, WAGE_DIST_MEAN + 4*WAGE_DIST_STD, 200)
wage_dist_pdf = norm.pdf(w_grid, loc=WAGE_DIST_MEAN, scale=WAGE_DIST_STD)
wage_dist_pdf /= np.sum(wage_dist_pdf)

def reservation_wage_objective(w_bar):
    integrand = np.maximum(w_grid, w_bar) * wage_dist_pdf
    integral = np.sum(integrand)
    v_continue = C + BETA * integral / (1 - BETA)
    v_stop = w_bar / (1 - BETA)
    return v_stop - v_continue

reservation_wage = fsolve(reservation_wage_objective, x0=10.0)[0]
note(f"The reservation wage is w* = ${reservation_wage:.2f}")

plt.figure(figsize=(10, 6))
plt.plot(w_grid, wage_dist_pdf, label='Wage Offer Distribution')
plt.axvline(reservation_wage, color='r', ls='--', label=f'Reservation Wage w*')
plt.fill_between(w_grid, wage_dist_pdf, where=w_grid < reservation_wage, color='red', alpha=0.2, label='Reject')
plt.fill_between(w_grid, wage_dist_pdf, where=w_grid >= reservation_wage, color='green', alpha=0.2, label='Accept')
plt.title('Optimal Policy in the McCall Search Model')
plt.xlabel('Wage Offer (w)'); plt.ylabel('Probability Density'); plt.legend()
plt.show()

### 2. The Diamond-Mortensen-Pissarides (DMP) Matching Model
The DMP model, for which its creators won the 2010 Nobel Prize, is the workhorse model of a frictional labor market. It describes an environment where unemployed workers search for jobs and firms post vacancies. The key friction is that this search-and-matching process takes time and is costly.

#### 2.1 The Model Environment and Bellman Equations
The model consists of a large number of identical workers and firms. The core components are:
- **Matching Function:** $M(u, v) = m u^\alpha v^{1-\alpha}$ determines the total number of new jobs formed as a function of the mass of unemployed workers, `u`, and vacant jobs, `v`.
- **Market Tightness:** $\theta = v/u$ is the key indicator of labor market conditions.
- **Job-finding rate for workers:** $p(\theta) = M/u = m\theta^{1-\alpha}$.
- **Vacancy-filling rate for firms:** $q(\theta) = M/v = m\theta^{-\alpha}$.
- **Value Functions:** The values for an unemployed worker ($U$), an employed worker ($W$), a vacant firm ($V$), and a filled job ($J$) are given by the following asset-pricing equations:
  $$ rU = b + p(\theta)(W - U) $$ 
  $$ rW = w + s(U - W) $$
  $$ rV = -\kappa + q(\theta)(J - V) $$
  $$ rJ = y - w + s(V - J) $$
  where:
    - $r$: The interest rate, which also serves as the agent's discount rate.
    - $b$: Unemployment benefits (the flow value of being unemployed).
    - $w$: The wage paid to the worker.
    - $s$: The exogenous rate at which jobs are destroyed.
    - $\kappa$: The per-period cost for a firm to maintain an open vacancy.
    - $y$: The productivity of a filled job.
- **Wage Determination:** The wage is determined by Nash bargaining. The total surplus of a match is $S = (W-U) + J$. The worker receives a fraction $\phi$ (their bargaining power) of this surplus, leading to the wage equation: $w = \phi(y + \kappa\theta) + (1-\phi)b$. Notice that the worker's wage depends on their outside option ($b$) and their share of the match surplus, which itself includes the firm's saved search costs ($\kappa\theta$ represents the expected cost to find a new worker).

#### 2.2 Solving for the General Equilibrium
The equilibrium of the model is a market tightness $\theta^*$ and a wage $w^*$ that satisfy three conditions simultaneously:
1.  **Job Creation (Free Entry):** Firms post vacancies until the value of a vacancy is driven to zero ($V=0$). This implies that the cost of posting a vacancy equals the expected profit from filling it: $\frac{\kappa}{q(\theta)} = J$.
2.  **Wage Curve:** The wage equation from Nash bargaining.
3.  **Surplus Sharing:** The value of a filled job, $J$, is consistent with the wage being paid.

Combining these conditions gives us a system of two equations in two unknowns, $\theta$ and $J$. We can solve this system numerically.

In [ ]:
sec("Solving the Full DMP Model Equilibrium")
# --- Model Parameters ---
# s: Job separation rate
# r: Interest rate (discount rate)
# kappa: Vacancy posting cost
# m: Matching efficiency
# alpha: Elasticity of the matching function w.r.t. unemployment
# phi: Worker's bargaining power
# y: Productivity of a filled job
# b: Unemployment benefits
s, r, kappa, m, alpha, phi, y, b = 0.02, 0.01, 0.1, 0.5, 0.5, 0.5, 1.0, 0.4

def solve_dmp(params):
    s, r, kappa, m, alpha, phi, y, b = params
    q = lambda theta: m * theta**(-alpha)
    p = lambda theta: m * theta**(1 - alpha)
    
    def system(vars):
        theta, J = vars
        # Eq 1: Job Creation (V=0)
        eq1 = kappa / q(theta) - J
        # Eq 2: Surplus definition
        w = phi * (y + kappa * theta) + (1 - phi) * b
        eq2 = (y - w) / (r + s) - J
        return [eq1, eq2]
    
    theta_star, J_star = root(system, [1.0, 1.0]).x
    w_star = phi * (y + kappa * theta_star) + (1 - phi) * b
    u_star = s / (s + p(theta_star))
    v_star = theta_star * u_star
    return {'tightness': theta_star, 'unemployment_rate': u_star, 
            'vacancy_rate': v_star, 'wage': w_star, 'job_value': J_star}

params = [s, r, kappa, m, alpha, phi, y, b]
eq = solve_dmp(params)
note(f"DMP Equilibrium found: Tightness θ*={eq['tightness']:.3f}, Unemployment u*={eq['unemployment_rate']:.2%}, Wage w*={eq['wage']:.3f}")

#### 2.3 The Beveridge Curve
Solving the steady-state flow equation, $p(\theta)u = s(1-u)$, gives the **Beveridge Curve**, a fundamental downward-sloping relationship between unemployment and vacancies ($v = \theta u$).

#### 2.4 Policy Implications
The DMP model provides a powerful lens for analyzing labor market policies. For instance:
- **Unemployment Benefits ($b$):** An increase in benefits raises the outside option of workers ($U$), allowing them to bargain for higher wages. This reduces the profitability of a job for a firm ($J$), leading firms to post fewer vacancies. The result is a lower market tightness ($\theta$) and a higher equilibrium unemployment rate.
- **Firing Costs:** Introducing firing costs (which increase the cost of job separation, $s$) has ambiguous effects. While it may reduce separations, it also makes firms more hesitant to hire, potentially lowering vacancy creation.
- **Matching Efficiency ($m$):** Policies that improve the matching process (e.g., better job boards, retraining programs) increase $m$, shifting the Beveridge curve inwards and leading to lower unemployment for any given level of vacancies.

In [ ]:
sec("The Beveridge Curve")
p_theta = lambda theta: m * theta**(1 - alpha)
u_ss = lambda theta: s / (s + p_theta(theta))
theta_grid = np.linspace(0.1, 5, 200)
u_bc = u_ss(theta_grid); v_bc = theta_grid * u_bc

plt.plot(u_bc * 100, v_bc * 100, lw=2.5)
plt.title('The Beveridge Curve'); plt.xlabel('Unemployment Rate (%)'); plt.ylabel('Vacancy Rate (%)')
plt.grid(True); plt.show()

### 3. On-the-Job Search and Wage Dispersion

The basic DMP model, while powerful, predicts a single wage for all identical workers, which is counter-factual. The **Burdett-Mortensen (1998)** model extends the search framework to include **on-the-job search**, meaning that employed workers can continue to search for better offers. This single, realistic assumption has a profound consequence: it generates a non-degenerate, equilibrium wage distribution, even with identical workers and firms.

The intuition is that firms, knowing that their workers might be poached by a competing offer, must decide on an optimal wage-posting strategy. A low-wage firm will have its workers hired away more frequently than a high-wage firm. This tension creates a continuous distribution of wages where firms trade off lower wage costs against shorter job durations.

In [ ]:
sec("The Burdett-Mortensen Equilibrium Wage Distribution")
# --- Model Parameters ---
# lambda0: The Poisson arrival rate of job offers for unemployed workers.
# lambda1: The Poisson arrival rate of job offers for employed workers.
# delta: The exogenous job separation rate.
# r: The interest rate (discount rate).
# b: Unemployment benefits.
# y: Productivity of a filled job.
lambda0, lambda1, delta, r, b, y = 1.0, 0.2, 0.05, 0.01, 0.5, 1.0
k = lambda1 / (r + delta)

# Solve for reservation wage w_R
w_R_obj = lambda w: b + (lambda0 * k / (1 + k)) * (y - w) - w
w_R = fsolve(w_R_obj, 0.6)[0]

# Define the CDF of the wage distribution G(w)
G = lambda w: ((1 + k) / (k * (y - b))) * (w - b)

w_vals = np.linspace(w_R, y, 100)
cdf_vals = G(w_vals)
pdf_vals = (1 + k) / (k * (y - b)) * np.ones_like(w_vals)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
ax1.plot(w_vals, cdf_vals)
ax1.set_title('Equilibrium Wage CDF, G(w)')
ax1.set_xlabel('Wage (w)'); ax1.set_ylabel('CDF')
ax2.plot(w_vals, pdf_vals)
ax2.set_title('Equilibrium Wage PDF, g(w)')
ax2.set_xlabel('Wage (w)'); ax2.set_ylabel('Density')
plt.suptitle('Burdett-Mortensen Wage Distribution', fontsize=16)
plt.show()

### 4. Assortative Matching
The theory of **assortative matching**, pioneered by Gary Becker, addresses the question of *who matches with whom* when agents are heterogeneous. The key concept is **supermodularity**. If the production function $F(k,h)$ (where $k$ is firm productivity and $h$ is worker skill) is supermodular, meaning $\frac{\partial^2 F}{\partial k \partial h} > 0$, then the marginal product of skill is higher at more productive firms. This creates an incentive for **Positive Assortative Matching (PAM)**, where high-skill workers match with high-productivity firms, as this maximizes the total surplus of the economy.

### 5. Chapter Summary
- **McCall Model:** A partial equilibrium model of optimal stopping that gives rise to the **reservation wage**.
- **DMP Model:** The workhorse general equilibrium model of a frictional labor market. The interaction of a matching function, free entry by firms, and wage bargaining determines the equilibrium unemployment and vacancy rates, which are linked by the **Beveridge Curve**.
- **Burdett-Mortensen Model:** The canonical model of on-the-job search. It shows how search frictions can lead to a non-degenerate **equilibrium wage distribution** even with identical agents and firms.
- **Assortative Matching:** Explains sorting patterns in markets with heterogeneous agents. If the production function is **supermodular**, we expect Positive Assortative Matching (high-types with high-types).

### 6. Exercises

1.  **Reservation Wage Dynamics:** In the McCall model, how would you expect the reservation wage to change if the wage offer distribution becomes riskier (i.e., has a higher variance but the same mean)? Explain the economic intuition and then verify it.

2.  **DMP Equilibrium Effects (Bargaining Power):** In the DMP model, solve for the full equilibrium. Then, analyze the effect of an increase in worker bargaining power `phi` (from 0.5 to 0.7) on the equilibrium market tightness `theta` and the unemployment rate `u`. Explain the mechanism.

3.  **Beveridge Curve Shifts:** Show that a decrease in matching efficiency `m` (from 0.5 to 0.4) causes the Beveridge curve to shift outwards (i.e., for any given vacancy rate, the unemployment rate is higher).

4.  **Burdett-Mortensen and Labor Market Policy:** In the Burdett-Mortensen model, what is the effect of an increase in unemployment benefits `b` on the reservation wage and the wage distribution? Does this policy increase or decrease wage inequality (e.g., the 90-10 wage ratio)?

5.  **Assortative Matching:** Consider a production function $F(k, h) = (k + h)^{0.5}$, where `k` is firm capital and `h` is worker human capital. Is this function supermodular or submodular? Would you expect positive or negative assortative matching in this economy? Verify your answer computationally.